In [1]:
#!pip install plotly --upgrade

import plotly.express as px
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import savgol_filter

In [2]:
spectra = loadmat('/content/drive/MyDrive/Spectra_3s_25c.mat')
spectra.keys()

dict_keys(['__header__', '__version__', '__globals__', 'AmpSAmp', 'Dm_total_save', 'GammaAmp', 'GammaSAmp', 'GammaS_total_save', 'Gamma_total_save', 'M', 'NmaxS', 'dG', 'dGS', 'frequencies', 'n_int', 'noiseAmpScaled', 'omegaS_total_save', 'omega_total_save'])

In [3]:
pd.options.plotting.backend = "plotly" # set default pandas plot to plotly

In [160]:
all_channel_prominence=0.09
density_window_size = 10
density_smoothness = 11
density_prominence = 0.5

In [161]:
# select a spectra (0,1, or 2)
spectra_number=1
modes = [x[spectra_number] for x in spectra['omega_total_save']]
all_channels = pd.DataFrame(np.transpose(np.transpose(spectra['Dm_total_save'])[spectra_number]))
all_channels.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.048936,0.070166,0.240180,0.086076,0.018956,0.090042,0.030591,0.010660,0.002478,0.004878,0.013486,0.023924,0.012981,0.119922,0.020306,0.006195,0.014207,0.002710,0.076716,0.068742,0.004098,0.031451,0.143340,0.090156,0.027009
1,0.035281,0.063212,0.226341,0.085063,0.009162,0.091403,0.039520,0.010418,0.002269,0.004719,0.011077,0.023279,0.005388,0.110728,0.026507,0.003151,0.009805,0.000605,0.080212,0.057247,0.015865,0.039005,0.146996,0.089339,0.035514
2,0.032246,0.064631,0.219429,0.077958,0.030805,0.097376,0.021813,0.010518,0.007309,0.003042,0.014385,0.021522,0.008172,0.120350,0.028783,0.004556,0.013486,0.004421,0.078757,0.055513,0.006246,0.061983,0.143205,0.082400,0.043331
3,0.047651,0.076099,0.227229,0.077916,0.013775,0.090301,0.036169,0.010615,0.002165,0.004050,0.002214,0.016837,0.009802,0.110040,0.025482,0.004558,0.006234,0.001716,0.064514,0.074167,0.016257,0.057225,0.147605,0.083171,0.029470
4,0.026209,0.074983,0.233617,0.080151,0.015280,0.091479,0.031783,0.010494,0.001918,0.005163,0.014173,0.014629,0.003730,0.108207,0.023055,0.008340,0.012219,0.003425,0.066602,0.064744,0.012959,0.054316,0.157985,0.087181,0.045813


In [162]:
# detect peaks across all channels
all_peaks=[]
for c in all_channels:
    peaks, properties = find_peaks(all_channels[c], prominence=all_channel_prominence)
    all_peaks.extend(peaks)
all_peaks=np.sort(all_peaks)
len(all_peaks)

82096

In [163]:
# create a density map of all peaks using window
half_size=int(density_window_size/2)
density = [0]*half_size
seeker=0
for i in range(half_size,all_channels.shape[0]-half_size):
  count = 0
  # count peaks
  while seeker<len(all_peaks) and all_peaks[seeker] < i+half_size:
    count += 1
    seeker += 1
  # return seeker to value half_size back
  if count > 0:
    while seeker>=0 and seeker < len(all_peaks) and all_peaks[seeker] > i-half_size:
      seeker-=1
    seeker = max(0, seeker)
  if count < np.shape(all_channels)[1]/10:
    count=0
  density.append(count)
len(density)

999995

In [164]:
smooth_density=savgol_filter(density, density_smoothness, 2)

In [165]:
peaks_detected_count = []
peaks_detected = []
for s in range(10):
  window_peaks, _ = find_peaks(smooth_density[s*100000 : (s+1)*100000], prominence=density_prominence)
  peaks_detected_count.append(len(window_peaks))
  peaks_detected.extend(window_peaks+s*100000)
print("number of peaks detected:", str(np.sum(peaks_detected_count)))

number of peaks detected: 10730


In [166]:
# distribution of detected peaks across each 10th
peaks_detected_count

[186, 516, 742, 951, 1073, 1208, 1381, 1481, 1580, 1612]

In [167]:
actual_mode_count = []
for s in range(10):
  actual_mode_count.append(len(pd.Series(modes)[pd.Series(modes)<(s+1)/10][pd.Series(modes)>s/10]))
actual_mode_count # actual mode distribution

[100, 300, 500, 700, 900, 1100, 1300, 1500, 1700, 1900]

In [168]:
# don't make the range too wide or it will take forever to run
view_range = (990000, 991000)
t_modes = np.sort([x*all_channels.shape[0] for x in modes])
temp_modes = pd.Series(t_modes)[(pd.Series(t_modes) < view_range[1])]
temp_modes = pd.Series(temp_modes)[(pd.Series(temp_modes) > view_range[0])]
temp_peaks = pd.Series(peaks_detected)[(pd.Series(peaks_detected) < view_range[1])]
temp_peaks = pd.Series(temp_peaks)[(pd.Series(temp_peaks) > view_range[0])]

In [169]:
# plot the predicted peaks, modes, and density
fig = pd.Series(smooth_density).iloc[range(view_range[0],view_range[1])].plot(template='plotly_dark', kind='line')
for mode in temp_modes:
    fig.add_vline(x=mode, line_width=1, line_color="green") #requires plotly 4.12 and above
for peak in temp_peaks:
    fig.add_vline(x=peak, line_width=1, line_dash="dash", line_color="red") #requires plotly 4.12 and above
fig.show()
# red is predicted
# green is actual modes

In [170]:
scaled_modes = []
for m in modes:
  scaled_modes.append(m*1000000)
  found_modes=[]

In [171]:
for i in scaled_modes:
    if (np.min(abs(peaks_detected - i)) < 50):
        found_modes.append(i)
len(found_modes)/10000

0.9556